In [ ]:
print('Setup complete.')

# Prompt Variants Demo

## Learning Objectives
- See multiple prompt approaches for the same task
- Compare effectiveness of different prompt strategies
- Understand when to use each variant type
- Learn systematic prompt variation techniques

## The Demo: One Task, Many Approaches

We'll demonstrate:
1. **Single Task Definition** - Email prioritization
2. **Multiple Variants** - Different prompt strategies
3. **Performance Comparison** - Which works best when
4. **Context Sensitivity** - How variants perform in different scenarios
5. **Selection Strategy** - Choosing the right variant

In [ ]:
# Setup and imports
!pip install asksageclient pip_system_certs
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import time
import tiktoken
from pathlib import Path
from typing import Dict, List, Any

# Import our AskSage client
from asksageclient import AskSageClient

# Get API credentials from Google Colab secrets
from google.colab import userdata
api_key = userdata.get('ASKSAGE_API_KEY')
email = userdata.get('ASKSAGE_EMAIL')

# Initialize client and tokenizer
client = AskSageClient(api_key=api_key, email=email)
tokenizer = tiktoken.encoding_for_model("gpt-4")
print("AskSage client initialized successfully")
print("Ready to showcase AI capabilities...")

## Task: Email Prioritization

We'll use email prioritization as our test case with multiple sample emails:

In [ ]:
# Test emails for variant comparison
test_emails = [
    {
        "subject": "URGENT: Production server down",
        "content": "Our main production server crashed at 2 PM. All customer transactions are failing. Revenue impact is $50K per hour. Need immediate attention.",
        "sender": "ops-team@company.com",
        "expected_priority": "Critical"
    },
    {
        "subject": "Weekly team meeting reminder",
        "content": "Just a reminder that our weekly team meeting is tomorrow at 10 AM in conference room B. Please bring your status updates.",
        "sender": "manager@company.com",
        "expected_priority": "Low"
    },
    {
        "subject": "Client contract needs review",
        "content": "The Johnson Industries contract is ready for legal review. They need it back by Friday for the board meeting. It's a $2M deal.",
        "sender": "sales@company.com",
        "expected_priority": "High"
    },
    {
        "subject": "New employee onboarding",
        "content": "Sarah Johnson starts Monday. Please prepare her workspace and send her the onboarding checklist. IT needs to set up her accounts.",
        "sender": "hr@company.com",
        "expected_priority": "Medium"
    },
    {
        "subject": "Security vulnerability detected",
        "content": "Our security scan found a critical vulnerability in the payment processing module. CVE-2024-1234. Patch available but requires downtime.",
        "sender": "security@company.com",
        "expected_priority": "Critical"
    }
]

print(f"Test dataset loaded: {len(test_emails)} emails")
print("Expected priorities defined for comparison")

## Variant 1: Direct Classification

Simple, straightforward approach:

In [ ]:
# Variant 1: Direct classification
def variant_1_direct(email):
    return f"""
Classify this email's priority as Critical, High, Medium, or Low.

Subject: {email['subject']}
From: {email['sender']}
Content: {email['content']}

Priority:
"""

print("=== VARIANT 1: DIRECT CLASSIFICATION ===")
variant_1_results = []

for i, email in enumerate(test_emails):
    prompt = variant_1_direct(email)
    
# Test GPT-5-mini
print("=== TESTING GPT-5-mini ===")
start_time = time.time()

response = client.query(
    message=prompt,
    system_prompt="You are concise.",
    temperature=0.1,
    model="gpt-5-mini",
    live=0,
    limit_references=0,
)

    
result = response.get("message").strip()
    variant_1_results.append({
        'email_id': i,
        'subject': email['subject'],
        'expected': email['expected_priority'],
        'actual': result,
        'variant': 'direct'
    })
    
    print(f"Email {i+1}: {email['subject'][:40]}...")
    print(f"Expected: {email['expected_priority']} | Got: {result}")
    print()

print(f"Variant 1 tested on {len(variant_1_results)} emails")

## Variant 2: Criteria-Based Classification

Provide explicit criteria for decision making:

In [ ]:
# Variant 2: Criteria-based classification
def variant_2_criteria(email):
    return f"""
Classify this email's priority using these criteria:

CRITICAL: System outages, security breaches, immediate revenue impact
HIGH: Important deadlines, large deals, executive requests
MEDIUM: Regular business operations, planning, coordination
LOW: FYI, routine updates, non-urgent reminders

Email to classify:
Subject: {email['subject']}
From: {email['sender']}
Content: {email['content']}

Analysis: [Explain which criteria apply]
Priority: [Critical/High/Medium/Low]
"""

print("=== VARIANT 2: CRITERIA-BASED CLASSIFICATION ===")
variant_2_results = []

for i, email in enumerate(test_emails):
    prompt = variant_2_criteria(email)
    
# Test GPT-5-mini
print("=== TESTING GPT-5-mini ===")
start_time = time.time()

response = client.query(
    message=prompt,
    system_prompt="You are concise.",
    temperature=0.1,
    model="gpt-5-mini",
    live=0,
    limit_references=0,
)

    
result = response.get("message").strip()
result = len(tokenizer.encode(result))

    # Extract just the priority
    priority_line = [line for line in result.split('\n') if 'Priority:' in line]
    priority = priority_line[0].split('Priority:')[-1].strip() if priority_line else result
    
    variant_2_results.append({
        'email_id': i,
        'subject': email['subject'],
        'expected': email['expected_priority'],
        'actual': priority,
        'full_response': result,
        'variant': 'criteria'
    })
    
    print(f"Email {i+1}: {email['subject'][:40]}...")
    print(f"Expected: {email['expected_priority']} | Got: {priority}")
    print(f"Analysis: {result.split('Analysis:')[-1].split('Priority:')[0].strip()[:100]}...")
    print()

print(f"Variant 2 tested on {len(variant_2_results)} emails")

## Variant 3: Scoring-Based Classification

Use numerical scoring for more precise classification:

In [ ]:
# Variant 3: Scoring-based classification
def variant_3_scoring(email):
    return f"""
Score this email on multiple dimensions (1-10 scale) and determine priority:

Email:
Subject: {email['subject']}
From: {email['sender']}
Content: {email['content']}

Score each dimension:
Urgency (1-10): [How time-sensitive is this?]
Impact (1-10): [What's the business impact?]
Effort (1-10): [How much work is required?]
Sender Authority (1-10): [How important is the sender?]

Total Score: [Sum of scores]
Priority Mapping: 35-40=Critical, 25-34=High, 15-24=Medium, 4-14=Low
Final Priority: [Based on total score]
"""

print("=== VARIANT 3: SCORING-BASED CLASSIFICATION ===")
variant_3_results = []

for i, email in enumerate(test_emails):
    prompt = variant_3_scoring(email)
    
# Test GPT-5-mini
print("=== TESTING GPT-5-mini ===")
start_time = time.time()

response = client.query(
    message=prompt,
    system_prompt="You are concise.",
    temperature=0.1,
    model="gpt-5-mini",
    live=0,
    limit_references=0,
)

    
result = response.get("message").strip()
result = len(tokenizer.encode(result))

    # Extract priority
    priority_line = [line for line in result.split('\n') if 'Final Priority:' in line]
    priority = priority_line[0].split('Final Priority:')[-1].strip() if priority_line else "Unknown"
    
    # Extract total score
    score_line = [line for line in result.split('\n') if 'Total Score:' in line]
    score = score_line[0].split('Total Score:')[-1].strip() if score_line else "Unknown"
    
    variant_3_results.append({
        'email_id': i,
        'subject': email['subject'],
        'expected': email['expected_priority'],
        'actual': priority,
        'score': score,
        'full_response': result,
        'variant': 'scoring'
    })
    
    print(f"Email {i+1}: {email['subject'][:40]}...")
    print(f"Expected: {email['expected_priority']} | Got: {priority} (Score: {score})")
    print()

print(f"Variant 3 tested on {len(variant_3_results)} emails")

## Variant 4: Role-Playing Classification

Give AI a specific role and context:

In [ ]:
# Variant 4: Role-playing classification
def variant_4_roleplay(email):
    return f"""
You are Alex Chen, a Senior Executive Assistant with 8 years of experience managing C-level executives' communications. You're known for your excellent judgment in prioritizing emails and protecting your executive's time while ensuring critical issues get immediate attention.

Your executive is currently in back-to-back meetings until 5 PM and has asked you to prioritize their inbox. They trust your judgment completely.

Classify this email:
Subject: {email['subject']}
From: {email['sender']}
Content: {email['content']}

Your assessment: [Brief explanation of your reasoning]
Priority: [Critical/High/Medium/Low]
Recommended action: [What should the executive do?]
"""

print("=== VARIANT 4: ROLE-PLAYING CLASSIFICATION ===")
variant_4_results = []

for i, email in enumerate(test_emails):
    prompt = variant_4_roleplay(email)
    
# Test GPT-5-mini
print("=== TESTING GPT-5-mini ===")
start_time = time.time()

response = client.query(
    message=prompt,
    system_prompt="You are concise.",
    temperature=0.2,
    model="gpt-5-mini",
    live=0,
    limit_references=0,
)

    
result = response.get("message").strip()
result = len(tokenizer.encode(result))

    # Extract priority
    priority_line = [line for line in result.split('\n') if 'Priority:' in line]
    priority = priority_line[0].split('Priority:')[-1].strip() if priority_line else "Unknown"
    
    variant_4_results.append({
        'email_id': i,
        'subject': email['subject'],
        'expected': email['expected_priority'],
        'actual': priority,
        'full_response': result,
        'variant': 'roleplay'
    })
    
    print(f"Email {i+1}: {email['subject'][:40]}...")
    print(f"Expected: {email['expected_priority']} | Got: {priority}")
    print()

print(f"Variant 4 tested on {len(variant_4_results)} emails")

## Variant 5: JSON-Structured Classification

Force structured output with detailed reasoning:

In [ ]:
# Variant 5: JSON-structured classification
def variant_5_json(email):
    return f"""
Analyze this email and provide classification in the exact JSON format below:

Email:
Subject: {email['subject']}
From: {email['sender']}
Content: {email['content']}

Respond with valid JSON only:
{{
  "priority": "Critical|High|Medium|Low",
  "urgency_score": "number 1-10",
  "impact_score": "number 1-10",
  "key_factors": ["list of factors that influenced the decision"],
  "reasoning": "brief explanation",
  "recommended_response_time": "immediate|within 1 hour|within 4 hours|within 24 hours|when convenient",
  "confidence": "number 1-10"
}}
"""

print("=== VARIANT 5: JSON-STRUCTURED CLASSIFICATION ===")
variant_5_results = []

for i, email in enumerate(test_emails):
    prompt = variant_5_json(email)
    
# Test GPT-5-mini
print("=== TESTING GPT-5-mini ===")
start_time = time.time()

response = client.query(
    message=prompt,
    system_prompt="You are concise.",
    temperature=0.1,
    model="gpt-5-mini",
    live=0,
    limit_references=0,
)

    
result = response.get("message").strip()
result = len(tokenizer.encode(result))

    
    # Try to parse JSON
    try:
        import re
        json_match = re.search(r'\{.*\}', result, re.DOTALL)
        if json_match:
            json_data = json.loads(json_match.group())
            priority = json_data.get('priority', 'Unknown')
            confidence = json_data.get('confidence', 'Unknown')
            reasoning = json_data.get('reasoning', 'No reasoning provided')
        else:
            priority = "Parse Error"
            confidence = "Unknown"
            reasoning = "Could not parse JSON"
    except json.JSONDecodeError:
        priority = "JSON Error"
        confidence = "Unknown"
        reasoning = "Invalid JSON format"
    
    variant_5_results.append({
        'email_id': i,
        'subject': email['subject'],
        'expected': email['expected_priority'],
        'actual': priority,
        'confidence': confidence,
        'reasoning': reasoning,
        'full_response': result,
        'variant': 'json'
    })
    
    print(f"Email {i+1}: {email['subject'][:40]}...")
    print(f"Expected: {email['expected_priority']} | Got: {priority} (Confidence: {confidence})")
    print(f"Reasoning: {reasoning[:80]}...")
    print()

print(f"Variant 5 tested on {len(variant_5_results)} emails")

## Variant Performance Analysis

Let AI analyze which variants performed best:

In [ ]:
# Compile all results for analysis
all_results = {
    "variant_1_direct": variant_1_results,
    "variant_2_criteria": variant_2_results,
    "variant_3_scoring": variant_3_results,
    "variant_4_roleplay": variant_4_results,
    "variant_5_json": variant_5_results
}

# AI analyzes variant performance
analysis_prompt = f"""
Analyze the performance of these 5 prompt variants for email prioritization:

TEST RESULTS:
{json.dumps(all_results, indent=2)}

Provide comprehensive analysis:
{{
  "accuracy_comparison": {{
    "variant_1_direct": "accuracy percentage",
    "variant_2_criteria": "accuracy percentage",
    "variant_3_scoring": "accuracy percentage",
    "variant_4_roleplay": "accuracy percentage",
    "variant_5_json": "accuracy percentage"
  }},
  "best_performing_variant": "variant name",
  "variant_strengths": {{
    "variant_1_direct": ["list of strengths"],
    "variant_2_criteria": ["list of strengths"],
    "variant_3_scoring": ["list of strengths"],
    "variant_4_roleplay": ["list of strengths"],
    "variant_5_json": ["list of strengths"]
  }},
  "use_case_recommendations": [
    {{
      "scenario": "string",
      "recommended_variant": "string",
      "rationale": "string"
    }}
  ],
  "key_insights": ["list of insights about prompt variants"]
}}
"""

print("=== VARIANT PERFORMANCE ANALYSIS ===")
# Test GPT-5-mini
print("=== TESTING GPT-5-mini ===")
start_time = time.time()

analysis_response = client.query(
    message=analysis_prompt,
    system_prompt="You are concise.",
    temperature=0.1,
    model="gpt-5-mini",
    live=0,
    limit_references=0,
)


analysis_result = analysis_response.get("message").strip()
print(analysis_result)

# Parse analysis results
import re
json_match = re.search(r'\{.*\}', analysis_result, re.DOTALL)
if json_match:
    analysis_data = json.loads(json_match.group())
    
    accuracy = analysis_data.get('accuracy_comparison', {})
    best_variant = analysis_data.get('best_performing_variant', 'Unknown')
    recommendations = analysis_data.get('use_case_recommendations', [])
    
    print(f"\n✓ Best performing variant: {best_variant}")
    print(f"✓ Use case recommendations: {len(recommendations)}")
    
    print("\nAccuracy Comparison:")
    for variant, acc in accuracy.items():
        print(f"  {variant}: {acc}")
    
    print("\nUse Case Recommendations:")
    for rec in recommendations:
        print(f"  {rec.get('scenario', 'Unknown scenario')}: Use {rec.get('recommended_variant', 'Unknown')}")
        print(f"    Rationale: {rec.get('rationale', 'No rationale provided')}")

## Prompt Variants Summary

### What We Demonstrated:

**1. Multiple Approaches, Same Task**
- Direct classification: Simple and fast
- Criteria-based: Explicit decision framework
- Scoring-based: Quantitative assessment
- Role-playing: Context and personality
- JSON-structured: Consistent, parseable output

**2. Performance Comparison**
- Accuracy varies by approach
- Different variants excel in different scenarios
- Trade-offs between speed, accuracy, and detail

**3. Context Sensitivity**
- Some variants better for complex decisions
- Others better for high-volume processing
- Role-playing adds human-like reasoning

### Key Insights:

**Variant Selection Strategy**
- **Speed Priority**: Use direct classification
- **Accuracy Priority**: Use criteria-based or scoring
- **Consistency Priority**: Use JSON-structured
- **Human-like Reasoning**: Use role-playing
- **Complex Decisions**: Use scoring-based

**When to Use Each Variant:**
- **High Volume**: Direct or JSON variants
- **Critical Decisions**: Criteria or scoring variants
- **User-Facing**: Role-playing variant
- **System Integration**: JSON variant
- **Training/Learning**: Criteria variant

### Best Practices:
- **Test Multiple Variants**: Don't assume one approach fits all
- **Measure Performance**: Use objective metrics
- **Consider Context**: Match variant to use case
- **Iterate and Improve**: Refine based on results
- **Document Decisions**: Track what works when

This demonstrates that prompt engineering is not one-size-fits-all - different approaches work better for different scenarios, and systematic testing helps identify the optimal strategy.